- k_v1-0-0: Bert_v1_0 x5fold, relative rank
- k_v1-1-0: Bert_v1_0(ep5) x5fold, relative rank
- k_v1-1-1: k_v1-1-0 + rule base for question_type_spelling
- k_v1-1-3: k_v1-1-0 + rule base for question_type_spelling(v2)

- Fork of k_v1-1-3: k_v1-1-3 + Bert_v1_0(tg20,21,22,23,24,25,29)
- k_v1-2-1: Fork of k_v1-1-3 + Bert_v1_0_classify(tg12,14,15)(numclass=4)
- k_v1-2-3: k_v1-2-1 + Bert_v1_0_classify(tg13)(numclass=5)
- k_v1-2-4: k_v1-2-3 + Bert_v1_0(tg0,1,3,5,6,7,16)(numclass=5)
- k_v1-2-4-1: k_v1-2-4 + postprocessing(to 0 or 1 using ave+std thresh)
- k_v1-2-4-2: k_v1-2-4 + postprocessing(to 0 or 1 using ave+std thresh) - Bert_v1_0_classify(tg12,14,15)(numclass=4)
- k_v1-2-4-3: k_v1-2-4-2 + ensemble(except tg13)
- k_v1-2-4-4: k_v1-2-4-3 - Bert_v1_0(ep5) + Bert_v1_2
- k_v1-2-4-7: k_v1-2-4-4 + ensemble(Bert_v1_0(msk))

In [1]:
from transformers import BertModel, BertTokenizer
import torch
import torch.nn as nn

import pandas as pd
import numpy as np
from sklearn.model_selection import GroupKFold
from scipy import stats
from scipy.stats import spearmanr

#from tqdm import tqdm
from tqdm.notebook import tqdm
from math import floor, ceil

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/google-quest-challenge/train.csv
/kaggle/input/google-quest-challenge/test.csv
/kaggle/input/google-quest-challenge/sample_submission.csv
/kaggle/input/qa-model/20020701_bert_v1_0_msk/Bert_v1_0_model_fold2
/kaggle/input/qa-model/20020701_bert_v1_0_msk/Bert_v1_0_model_fold4
/kaggle/input/qa-model/20020701_bert_v1_0_msk/Bert_v1_0_model_fold3
/kaggle/input/qa-model/20020701_bert_v1_0_msk/Bert_v1_0_model_fold1
/kaggle/input/qa-model/20020701_bert_v1_0_msk/log.txt
/kaggle/input/qa-model/20020701_bert_v1_0_msk/Bert_v1_0_model_fold0
/kaggle/input/qa-model/20020802_bert_v1_0_classify_comb1/Bert_v1_0_clasify_comb1_model_fold1
/kaggle/input/qa-model/20020802_bert_v1_0_classify_comb1/tg121415.txt
/kaggle/input/qa-model/20020802_bert_v1_0_classify_comb1/Bert_v1_0_clasify_comb1_model_fold2
/kaggle/input/qa-model/20020802_bert_v1_0_classify_comb1/Bert_v1_0_clasify_comb1_model_fold0
/kaggle/input/qa-model/transformers/bert-base-uncased/tokenizer/vocab.txt
/kaggle/input/qa-model/transfor

In [2]:
class config:
    PATH = '../input/google-quest-challenge/'

    BERT_PRETRAINED_MODEL = 'bert-base-uncased'
    BERT_TOKENIZER_PATH = '../input/qa-model/transformers/' + BERT_PRETRAINED_MODEL + '/tokenizer/'
    BERT_PRETRAINED_MODEL_PATH = '../input/qa-model/transformers/' + BERT_PRETRAINED_MODEL + '/pretrained_model/'

    #MY_MODEL = '../input/qa-model/mymodel/20020302_bert_v1_0/'
    #MY_MODEL = '../input/qa-model/20020302_bert_v1_0_ep5/'
    MY_MODEL = '../input/qa-model/20021001_bert_v1_2/'

    MY_MODEL1 = '../input/qa-model/20020701_bert_v1_0_msk/'
    
    MY_MODEL2 = '../input/qa-model/20020801_bert_v1_0_comb2/' # tg20,21,22,23,24,25,29
    TG_MY_MODEL2 = [20,21,22,23,24,25,29]
    
    #MY_MODEL3_CLASS = '../input/qa-model/20020802_bert_v1_0_classify_comb1/' # tg12,14,15
    #TG_MY_MODEL3_CLASS = [12,14,15] # tg12,14,15
    
    MY_MODEL4_CLASS = '../input/qa-model/20020803_bert_v1_0_classify_comb2/' # tg13
    TG_MY_MODEL4_CLASS = [13] # tg13
    NUMCLASS_MODEL4_CLASS = 5
    
    MY_MODEL5 = '../input/qa-model/20020801_bert_v1_0_comb1/' # tg0,1,3,5,6,7,16
    TG_MY_MODEL5 = [0,1,3,5,6,7,16]
    
    POSTPROC_TO_0_INDEX = [2, 9, 12, 14, 15]
    POSTPROC_TO_0_C = [1.2, 1.8, 1.6, 1.4, 1.6]
    POSTPROC_TO_1_INDEX = [5, ]
    POSTPROC_TO_1_C = [0.4, ]
    
    
class InputData:
    @staticmethod
    def get_train_data(to_relative_rank=False, clip_output=None):
        df = pd.read_csv(config.PATH+'train.csv')

        if to_relative_rank:
            df.iloc[:,11:] = (df.iloc[:,11:].apply(stats.mstats.rankdata, axis=0) - 0.5) / len(df.iloc[:,11:])

        if clip_output is not None:
            df.iloc[:,11:] = df.iloc[:,11:].clip(clip_output[0], clip_output[1])

        return df

    @staticmethod
    def get_test_data():
        return pd.read_csv(config.PATH+'test.csv')

class Submission:
    @staticmethod
    def get_submission_file():
        return pd.read_csv(config.PATH+'sample_submission.csv')

    @staticmethod
    def make_submission(pred, filename='submission.csv'):
        df_sub = Submission.get_submission_file()
        df_sub.iloc[:, 1:] = pred
        df_sub.to_csv(filename, index=False)
        return

class BertData_v3:
    num_special_token = 3

    @staticmethod
    def _get_masks(tokens, max_seq_length):
        if len(tokens)>max_seq_length:
            raise IndexError("Token length more than max seq length!")
        return [1]*len(tokens) + [0] * (max_seq_length - len(tokens))

    @staticmethod
    def _get_segments(tokens, max_seq_length):
        """Segments: 0 for the first sequence, 1 for the second"""
        if len(tokens)>max_seq_length:
            raise IndexError("Token length more than max seq length!")
        segments = []
        first_sep = True
        current_segment_id = 0
        for token in tokens:
            segments.append(current_segment_id)
            if token == "[SEP]":
                if first_sep:
                    first_sep = False 
                else:
                    current_segment_id = 1
        return segments + [0] * (max_seq_length - len(tokens))

    @staticmethod
    def _get_ids(tokens, tokenizer, max_seq_length):
        """Token ids from Tokenizer vocab"""
        token_ids = tokenizer.encode(tokens, add_special_tokens=False)
        input_ids = token_ids + [0] * (max_seq_length-len(token_ids))
        return input_ids

    @staticmethod
    def _trim_input(tokenizer, title, body, max_sequence_length, t_max_len=50):
        t = tokenizer.tokenize(title)
        b = tokenizer.tokenize(body)
    
        t_len = len(t)
        b_len = len(b)

        if (t_len + b_len + BertData_v3.num_special_token) > max_sequence_length:
            diff = (t_len + b_len + BertData_v3.num_special_token) - max_sequence_length

            if t_len > t_max_len:
                t_len_new = max(t_max_len, t_len - diff)
            else:
                t_len_new = t_len

            if (t_len_new + b_len + BertData_v3.num_special_token) > max_sequence_length:
                b_len_new = max_sequence_length - t_len_new - BertData_v3.num_special_token
            else:
                b_len_new = b_len

            t = t[:t_len_new]
            b = b[:b_len_new]

        return t, b

    @staticmethod
    def _convert_to_bert_inputs(title, body, tokenizer, max_sequence_length):
        """Converts tokenized input to ids, masks and segments for BERT"""
    
        stoken = ["[CLS]"] + title + ["[SEP]"] + body + ["[SEP]"]

        input_ids = BertData_v3._get_ids(stoken, tokenizer, max_sequence_length)
        input_masks = BertData_v3._get_masks(stoken, max_sequence_length)
        input_segments = BertData_v3._get_segments(stoken, max_sequence_length)

        return [input_ids, input_masks, input_segments]

    @staticmethod
    def get_input_categories(df_train):
        return list(df_train.columns[[1,2,5]])

    @staticmethod
    def get_output_categories(df_train):
        return list(df_train.columns[11:])

    @staticmethod
    def compute_input_arrays(df, tokenizer, max_sequence_length):
        columns = BertData_v3.get_input_categories(df)

        input_q_ids, input_q_masks, input_q_segments = [], [], []
        input_a_ids, input_a_masks, input_a_segments = [], [], []

        counter = 0
        for _, instance in df[columns].iterrows():
            t, q, a = instance.question_title, instance.question_body, instance.answer

            t_token, q_token = BertData_v3._trim_input(tokenizer, t, q, max_sequence_length)
            ids, masks, segments = BertData_v3._convert_to_bert_inputs(t_token, q_token, tokenizer, max_sequence_length)
            input_q_ids.append(ids)
            input_q_masks.append(masks)
            input_q_segments.append(segments)
            
            t_token, a_token = BertData_v3._trim_input(tokenizer, t, a, max_sequence_length)
            ids, masks, segments = BertData_v3._convert_to_bert_inputs(t_token, a_token, tokenizer, max_sequence_length)
            input_a_ids.append(ids)
            input_a_masks.append(masks)
            input_a_segments.append(segments)
            
        return [
                torch.tensor(np.asarray(input_q_ids, dtype=np.int64)), 
                torch.tensor(np.asarray(input_q_masks, dtype=np.int64)), 
                torch.tensor(np.asarray(input_q_segments, dtype=np.int64)),
                torch.tensor(np.asarray(input_a_ids, dtype=np.int64)), 
                torch.tensor(np.asarray(input_a_masks, dtype=np.int64)), 
                torch.tensor(np.asarray(input_a_segments, dtype=np.int64)),
                ]

    @staticmethod
    def compute_output_arrays(df):
        columns = BertData_v3.get_output_categories(df)
        return torch.tensor(np.asarray(df[columns]))

class BertUtils:
    @staticmethod
    def save_bert_tokenizer(pretrained_model, save_path):
        tokenizer = BertTokenizer.from_pretrained(pretrained_model)
        tokenizer.save_pretrained(save_path)
        return

    @staticmethod
    def get_bert_tokenizer(save_path):
        tokenizer = BertTokenizer.from_pretrained(save_path)
        return tokenizer

    @staticmethod
    def save_bert_model(pretrained_model, save_path):
        model = BertModel.from_pretrained(pretrained_model)
        model.save_pretrained(save_path)
        return

    @staticmethod
    def get_bert_model(save_path):
        model = BertModel.from_pretrained(save_path)
        return model

class Bert_v1_0(nn.Module):
    def __init__(self, pretrained_model, num_bert_last_hidden=768, num_q_target=21, num_a_target=9, drop_p=0.1):
        super(Bert_v1_0, self).__init__()

        self.pretrained_model = pretrained_model
        self.dropout = nn.Dropout(p=drop_p)
        self.q_linear = nn.Linear(num_bert_last_hidden, num_q_target)
        self.a_linear = nn.Linear(num_bert_last_hidden*2, num_a_target)

    def forward(self, q_id, q_mask, q_seg, a_id, a_mask, a_seg):
        oup = self.pretrained_model(q_id, q_mask, q_seg) # (batch, seq length, self.num_bert_last_hidden)
        oup = oup[0]
        q_hidden_oup = torch.mean(oup, dim=1)
        q_oup = self.dropout(q_hidden_oup)

        oup = self.pretrained_model(a_id, a_mask, a_seg) # (batch, seq length, self.num_bert_last_hidden)
        oup = oup[0]
        a_hidden_oup = torch.mean(oup, dim=1)
        a_oup = self.dropout(a_hidden_oup)

        # q:21, a:9
        q_result = self.q_linear(q_oup)
        a_result = self.a_linear(torch.cat([q_oup, a_oup], dim=1))

        return torch.cat([q_result, a_result], dim=1), torch.cat([q_hidden_oup, a_hidden_oup], dim=1)

    def freeze_pretrained_model(self):
        for param in self.pretrained_model.parameters():
            param.requires_grad = False
        return

class Bert_v1_0_classify(nn.Module):
    def __init__(self, pretrained_model, num_class=4, num_bert_last_hidden=768, num_q_target=21, num_a_target=9, drop_p=0.1):
        super(Bert_v1_0_classify, self).__init__()

        self.pretrained_model = pretrained_model
        self.dropout = nn.Dropout(p=drop_p)

        self.q_linears = nn.ModuleList(nn.Linear(num_bert_last_hidden, num_class) for i in range(num_q_target))
        self.a_linears = nn.ModuleList(nn.Linear(num_bert_last_hidden*2, num_class) for i in range(num_a_target))

    def forward(self, q_id, q_mask, q_seg, a_id, a_mask, a_seg):
        # q out
        oup = self.pretrained_model(q_id, q_mask, q_seg) # (batch, seq length, self.num_bert_last_hidden)
        oup = oup[0]
        q_hidden_oup = torch.mean(oup, dim=1)
        q_oup = self.dropout(q_hidden_oup)

        # a out
        oup = self.pretrained_model(a_id, a_mask, a_seg) # (batch, seq length, self.num_bert_last_hidden)
        oup = oup[0]
        a_hidden_oup = torch.mean(oup, dim=1)
        a_oup = self.dropout(a_hidden_oup)

        # results [(batch, class), (batch, class), ...]
        results = []
        # q:21
        for q_linear in self.q_linears:
            results.append(torch.unsqueeze(q_linear(q_oup), 1))
        # a:9
        for a_linear in self.a_linears:
            results.append(torch.unsqueeze(a_linear(torch.cat([q_oup, a_oup], dim=1)), 1))
        
        return torch.cat(results, dim=1), torch.cat([q_hidden_oup, a_hidden_oup], dim=1)

    def freeze_pretrained_model(self):
        for param in self.pretrained_model.parameters():
            param.requires_grad = False
        return
    
class Bert_v1_2(nn.Module):
    def __init__(self, pretrained_model, num_bert_last_hidden=768, num_q_target=21, num_a_target=9, drop_p=0.1):
        super(Bert_v1_2, self).__init__()

        self.pretrained_model = pretrained_model
        self.dropout = nn.Dropout(p=drop_p)
        self.q_linear = nn.Linear(num_bert_last_hidden, num_q_target)
        self.a_linear = nn.Linear(num_bert_last_hidden, num_a_target)

    def forward(self, q_id, q_mask, q_seg, a_id, a_mask, a_seg):
        oup = self.pretrained_model(q_id, q_mask, q_seg) # (batch, seq length, self.num_bert_last_hidden)
        oup = oup[0]
        q_hidden_oup = torch.mean(oup, dim=1)
        q_oup = self.dropout(q_hidden_oup)

        oup = self.pretrained_model(a_id, a_mask, a_seg) # (batch, seq length, self.num_bert_last_hidden)
        oup = oup[0]
        a_hidden_oup = torch.mean(oup, dim=1)
        a_oup = self.dropout(a_hidden_oup)

        # q:21, a:9
        q_result = self.q_linear(q_oup)
        a_result = self.a_linear(a_oup)

        return torch.cat([q_result, a_result], dim=1), torch.cat([q_hidden_oup, a_hidden_oup], dim=1)
    
class QADataset_SeparateQA(torch.utils.data.Dataset):
    def __init__(self, q_ids, q_masks, q_segments, a_ids, a_masks, a_segments, labels=None):
        self.q_ids = q_ids
        self.q_masks = q_masks
        self.q_segments = q_segments
        self.a_ids = a_ids
        self.a_masks = a_masks
        self.a_segments = a_segments
        self.labels = labels

    def __getitem__(self, idx):
        if self.labels is not None:
            return self.q_ids[idx], self.q_masks[idx], self.q_segments[idx], self.a_ids[idx], self.a_masks[idx], self.a_segments[idx], self.labels[idx]
        else:
            return self.q_ids[idx], self.q_masks[idx], self.q_segments[idx], self.a_ids[idx], self.a_masks[idx], self.a_segments[idx]

    def __len__(self):
        return len(self.q_ids)

def get_dataloader(dataset, batch_size, shuffle=True):
    return torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

def compute_spearmanr(original, preds):
    scores = []
    for i in range(30):
        scores.append(spearmanr(original[:, i], preds[:, i]).correlation)

    return np.nanmean(scores)

def sigmoid(x):
    y = 1 / (1 + np.exp(-x))
    return y

def calc_pred_Bert_v1_0(net, dataloader, with_target=False, classify=False):
    net = net.cuda()
    
    net.eval()

    preds = []
    original = []
    if with_target:
        with torch.no_grad():
            for batch_idx, (q_ids, q_masks, q_segments, a_ids, a_masks, a_segments, targets) in enumerate(tqdm(dataloader)):
                q_ids, q_masks, q_segments, targets = q_ids.cuda(), q_masks.cuda(), q_segments.cuda(), targets.cuda()
                a_ids, a_masks, a_segments = a_ids.cuda(), a_masks.cuda(), a_segments.cuda()
                outputs, hidden_outpus = net(q_ids, q_masks, q_segments, a_ids, a_masks, a_segments)

                if not classify:
                    preds.append(outputs.cpu().numpy())
                else:
                    preds.append(outputs.max(2)[1].cpu().numpy())
                original.append(targets.cpu().numpy())
    else:
        with torch.no_grad():
            for batch_idx, (q_ids, q_masks, q_segments, a_ids, a_masks, a_segments) in enumerate(tqdm(dataloader)):
                q_ids, q_masks, q_segments = q_ids.cuda(), q_masks.cuda(), q_segments.cuda()
                a_ids, a_masks, a_segments = a_ids.cuda(), a_masks.cuda(), a_segments.cuda()
                outputs, hidden_outpus = net(q_ids, q_masks, q_segments, a_ids, a_masks, a_segments)

                if not classify:
                    preds.append(outputs.cpu().numpy())
                else:
                    preds.append(outputs.max(2)[1].cpu().numpy())

    preds = np.concatenate(preds)
    preds = sigmoid(preds)

    if with_target:
        original = np.concatenate(original)    
        score = compute_spearmanr(original, preds)
        print('Score: %.5f' % (score,))

    return preds

def calc_pred_question_type_spelling(data_df):
    print('calc_pred_question_type_spelling')
    # category, host
    ex_x = data_df[data_df['category']=='CULTURE']
    print(len(ex_x))
    ex_x = ex_x[ex_x['host']=='english.stackexchange.com']
    print(len(ex_x))

    qts = ex_x['question_title']
    posi = []
    for qt in qts:
        words = qt.split(' ')
    #    or_flg = []
    #    or_flg.append('pronounce' in words)
    #    or_flg.append('pronunciation' in words)
    #    or_flg.append('sounds' in words)
    #    or_flg.append('sound' in words)
    #    or_flg.append('spell' in words)
    #    or_flg.append('spells' in words)
    #    or_flg.append('spelling' in words)
    #    or_flg.append('accent' in words)
    #    or_flg.append('accented' in words)
    #    or_flg.append('vowel' in words)
    #    or_flg.append('schwa' in words)
    #    or_flg.append('syllables' in words)
    #    or_flg.append('phoneme' in words)
    #    #or_flg.append('or' in words)
    #    #or_flg.append('and' in words)
    #        
        and_flg = []
        and_flg.append(not 'mean' in words)
        and_flg.append(not 'mean.' in words)
        and_flg.append(not 'mean?' in words)
        and_flg.append(not 'means' in words)
        and_flg.append(not 'means.' in words)
        and_flg.append(not 'means?' in words)
        and_flg.append(not 'meaning' in words)
        and_flg.append(not 'meaning.' in words)
        and_flg.append(not 'meaning?' in words)
        and_flg.append(not 'use' in words)
        and_flg.append(not 'use.' in words)
        and_flg.append(not 'use?' in words)
        and_flg.append(not 'uses' in words)
        and_flg.append(not 'uses.' in words)
        and_flg.append(not 'uses?' in words)
        and_flg.append(not 'using' in words)
        and_flg.append(not 'using.' in words)
        and_flg.append(not 'using?' in words)
        and_flg.append(not 'used' in words)
        and_flg.append(not 'used.' in words)
        and_flg.append(not 'used?' in words)

        #if any(or_flg) and all(and_flg):
        if all(and_flg):
            posi.append(1)
            print(qt)
        else:
            posi.append(0)

    posi = np.array(posi)    
    if np.all(posi < 0.5):
        print('all 0 in calc_pred_question_type_spelling')
        posi = 1

    pred = np.zeros(len(data_df))
    #pred[ex_x.index] = 1
    pred[ex_x.index] = posi

    return pred

In [3]:
def post_proc_to0or1(pred):
    # to 0
    for idx, c in zip(config.POSTPROC_TO_0_INDEX, config.POSTPROC_TO_0_C):
        tg = pred[:,idx].copy()
        cond = tg < np.average(tg) + np.std(tg) * c
        pred[cond,idx] = 0.00001
        
    # to 1
    for idx, c in zip(config.POSTPROC_TO_1_INDEX, config.POSTPROC_TO_1_C):
        tg = pred[:,idx].copy()
        cond = tg > np.average(tg) - np.std(tg) * c
        pred[cond,idx] = 0.99999
        
    return pred

In [4]:
def run_20020401():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold2'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold3'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold4'),
        ]

    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))

    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    # prediction
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [5]:
def run_20020601():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold2'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold3'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold4'),
        ]

    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))

    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    # prediction
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    # predcition by rule base
    pred_question_type_spelling = calc_pred_question_type_spelling(test_df)
    pred[:,19] = pred_question_type_spelling

    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

        # predcition by rule base
        pred_question_type_spelling = calc_pred_question_type_spelling(train_df)
        pred[:,19] = pred_question_type_spelling
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [6]:
def run_200208801():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold2'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold3'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold4'),
        ]
    
    model_path2 = [
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold0'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold1'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold2'),
        ]

    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))

    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    
    # prediction
    print('predict')
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    
    # prediction2
    print('predict2')
    pred2 = None
    counter = 0
    for mp in model_path2:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred2 is None:
            pred2 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred2 = pred2 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred2 = pred2 / counter
    
    pred[:,config.TG_MY_MODEL2] = pred2[:,config.TG_MY_MODEL2]
    
    # predcition by rule base
    print('predcition by rule base')
    pred_question_type_spelling = calc_pred_question_type_spelling(test_df)
    pred[:,19] = pred_question_type_spelling

    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

        # predcition by rule base
        pred_question_type_spelling = calc_pred_question_type_spelling(train_df)
        pred[:,19] = pred_question_type_spelling
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [7]:
def run_200208802():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold2'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold3'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold4'),
        ]
    
    model_path2 = [
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold0'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold1'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold2'),
        ]
    
    model_path3_class = [
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold0'),
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold1'),
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold2'),
        ]

    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))

    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    
    # prediction
    print('predict')
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    
    # prediction2
    print('predict2')
    pred2 = None
    counter = 0
    for mp in model_path2:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred2 is None:
            pred2 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred2 = pred2 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred2 = pred2 / counter
    
    pred[:,config.TG_MY_MODEL2] = pred2[:,config.TG_MY_MODEL2]
    
    # prediction3
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path))
    print('predict3')
    pred3 = None
    counter = 0
    for mp in model_path3_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred3 is None:
            pred3 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred3 = pred3 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred3 = pred3 / counter
    pred3 = pred3 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL3_CLASS] = pred3[:,config.TG_MY_MODEL3_CLASS]
    
    # predcition by rule base
    print('predcition by rule base')
    pred_question_type_spelling = calc_pred_question_type_spelling(test_df)
    pred[:,19] = pred_question_type_spelling

    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

        # predcition by rule base
        pred_question_type_spelling = calc_pred_question_type_spelling(train_df)
        pred[:,19] = pred_question_type_spelling
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [8]:
def run_200208803():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold2'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold3'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold4'),
        ]
    
    model_path2 = [
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold0'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold1'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold2'),
        ]
    
    model_path3_class = [
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold0'),
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold1'),
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold2'),
        ]
    
    model_path4_class = [
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold0'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold1'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold2'),
        ]

    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))

    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    
    # prediction
    print('predict')
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    
    # prediction2
    print('predict2')
    pred2 = None
    counter = 0
    for mp in model_path2:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred2 is None:
            pred2 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred2 = pred2 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred2 = pred2 / counter
    
    pred[:,config.TG_MY_MODEL2] = pred2[:,config.TG_MY_MODEL2]
    
    # prediction3
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path))
    print('predict3')
    pred3 = None
    counter = 0
    for mp in model_path3_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred3 is None:
            pred3 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred3 = pred3 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred3 = pred3 / counter
    pred3 = pred3 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL3_CLASS] = pred3[:,config.TG_MY_MODEL3_CLASS]
    
    
    # prediction4
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path), num_class=config.NUMCLASS_MODEL4_CLASS)
    print('predict4')
    pred4 = None
    counter = 0
    for mp in model_path4_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred4 is None:
            pred4 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred4 = pred4 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred4 = pred4 / counter
    pred4 = pred4 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL4_CLASS] = pred4[:,config.TG_MY_MODEL4_CLASS]
    
    
    # predcition by rule base
    print('predcition by rule base')
    pred_question_type_spelling = calc_pred_question_type_spelling(test_df)
    pred[:,19] = pred_question_type_spelling

    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

        # predcition by rule base
        pred_question_type_spelling = calc_pred_question_type_spelling(train_df)
        pred[:,19] = pred_question_type_spelling
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [9]:
def run_20021001():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold2'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold3'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold4'),
        ]
    
    model_path2 = [
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold0'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold1'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold2'),
        ]
    
    model_path3_class = [
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold0'),
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold1'),
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold2'),
        ]
    
    model_path4_class = [
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold0'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold1'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold2'),
        ]
    
    model_path5 = [
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold0'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold1'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold2'),
        ]

    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))

    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    
    # prediction
    print('predict')
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    
    # prediction2
    print('predict2')
    pred2 = None
    counter = 0
    for mp in model_path2:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred2 is None:
            pred2 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred2 = pred2 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred2 = pred2 / counter
    
    pred[:,config.TG_MY_MODEL2] = pred2[:,config.TG_MY_MODEL2]
    
    # prediction3
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path))
    print('predict3')
    pred3 = None
    counter = 0
    for mp in model_path3_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred3 is None:
            pred3 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred3 = pred3 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred3 = pred3 / counter
    pred3 = pred3 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL3_CLASS] = pred3[:,config.TG_MY_MODEL3_CLASS]
    
    
    # prediction4
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path), num_class=config.NUMCLASS_MODEL4_CLASS)
    print('predict4')
    pred4 = None
    counter = 0
    for mp in model_path4_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred4 is None:
            pred4 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred4 = pred4 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred4 = pred4 / counter
    pred4 = pred4 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL4_CLASS] = pred4[:,config.TG_MY_MODEL4_CLASS]
    
    # prediction5
    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))
    print('predict5')
    pred5 = None
    counter = 0
    for mp in model_path5:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred5 is None:
            pred5 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred5 = pred5 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred5 = pred5 / counter
    
    pred[:,config.TG_MY_MODEL5] = pred5[:,config.TG_MY_MODEL5]
    
    
    # predcition by rule base
    print('predcition by rule base')
    pred_question_type_spelling = calc_pred_question_type_spelling(test_df)
    pred[:,19] = pred_question_type_spelling

    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

        # predcition by rule base
        pred_question_type_spelling = calc_pred_question_type_spelling(train_df)
        pred[:,19] = pred_question_type_spelling
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [10]:
def run_20021002():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold2'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold3'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold4'),
        ]
    
    model_path2 = [
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold0'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold1'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold2'),
        ]
    
    model_path3_class = [
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold0'),
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold1'),
        os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold2'),
        ]
    
    model_path4_class = [
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold0'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold1'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold2'),
        ]
    
    model_path5 = [
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold0'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold1'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold2'),
        ]

    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))

    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    
    # prediction
    print('predict')
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    
    # prediction2
    print('predict2')
    pred2 = None
    counter = 0
    for mp in model_path2:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred2 is None:
            pred2 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred2 = pred2 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred2 = pred2 / counter
    
    pred[:,config.TG_MY_MODEL2] = pred2[:,config.TG_MY_MODEL2]
    
    # prediction3
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path))
    print('predict3')
    pred3 = None
    counter = 0
    for mp in model_path3_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred3 is None:
            pred3 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred3 = pred3 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred3 = pred3 / counter
    pred3 = pred3 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL3_CLASS] = pred3[:,config.TG_MY_MODEL3_CLASS]
    
    
    # prediction4
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path), num_class=config.NUMCLASS_MODEL4_CLASS)
    print('predict4')
    pred4 = None
    counter = 0
    for mp in model_path4_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred4 is None:
            pred4 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred4 = pred4 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred4 = pred4 / counter
    pred4 = pred4 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL4_CLASS] = pred4[:,config.TG_MY_MODEL4_CLASS]
    
    # prediction5
    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))
    print('predict5')
    pred5 = None
    counter = 0
    for mp in model_path5:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred5 is None:
            pred5 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred5 = pred5 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred5 = pred5 / counter
    
    pred[:,config.TG_MY_MODEL5] = pred5[:,config.TG_MY_MODEL5]
    
    
    # predcition by rule base
    print('predcition by rule base')
    pred_question_type_spelling = calc_pred_question_type_spelling(test_df)
    pred[:,19] = pred_question_type_spelling

    # post processing to 0 or 1
    print('post processing to 0 or 1')
    pred = post_proc_to0or1(pred)
    
    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

        # predcition by rule base
        pred_question_type_spelling = calc_pred_question_type_spelling(train_df)
        pred[:,19] = pred_question_type_spelling
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [11]:
def run_20021003():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold2'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold3'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold4'),
        ]
    
    model_path2 = [
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold0'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold1'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold2'),
        ]
    
    #model_path3_class = [
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold0'),
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold1'),
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold2'),
    #    ]
    
    model_path4_class = [
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold0'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold1'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold2'),
        ]
    
    model_path5 = [
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold0'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold1'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold2'),
        ]

    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))

    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    
    # prediction
    print('predict')
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    
    # prediction2
    print('predict2')
    pred2 = None
    counter = 0
    for mp in model_path2:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred2 is None:
            pred2 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred2 = pred2 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred2 = pred2 / counter
    
    pred[:,config.TG_MY_MODEL2] = pred2[:,config.TG_MY_MODEL2]
    
    """
    # prediction3
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path))
    print('predict3')
    pred3 = None
    counter = 0
    for mp in model_path3_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred3 is None:
            pred3 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred3 = pred3 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred3 = pred3 / counter
    pred3 = pred3 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL3_CLASS] = pred3[:,config.TG_MY_MODEL3_CLASS]
    """
    
    # prediction4
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path), num_class=config.NUMCLASS_MODEL4_CLASS)
    print('predict4')
    pred4 = None
    counter = 0
    for mp in model_path4_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred4 is None:
            pred4 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred4 = pred4 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred4 = pred4 / counter
    pred4 = pred4 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL4_CLASS] = pred4[:,config.TG_MY_MODEL4_CLASS]
    
    # prediction5
    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))
    print('predict5')
    pred5 = None
    counter = 0
    for mp in model_path5:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred5 is None:
            pred5 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred5 = pred5 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred5 = pred5 / counter
    
    pred[:,config.TG_MY_MODEL5] = pred5[:,config.TG_MY_MODEL5]
    
    
    # predcition by rule base
    print('predcition by rule base')
    pred_question_type_spelling = calc_pred_question_type_spelling(test_df)
    pred[:,19] = pred_question_type_spelling

    # post processing to 0 or 1
    print('post processing to 0 or 1')
    pred = post_proc_to0or1(pred)
    
    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

        # predcition by rule base
        pred_question_type_spelling = calc_pred_question_type_spelling(train_df)
        pred[:,19] = pred_question_type_spelling
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [12]:
def run_20021101():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold2'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold3'),
        os.path.join(config.MY_MODEL, 'Bert_v1_0_model_fold4'),
        ]
    
    model_path2 = [
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold0'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold1'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold2'),
        ]
    
    #model_path3_class = [
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold0'),
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold1'),
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold2'),
    #    ]
    
    model_path4_class = [
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold0'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold1'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold2'),
        ]
    
    model_path5 = [
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold0'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold1'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold2'),
        ]

    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))

    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    
    # prediction
    print('predict')
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    
    # prediction2
    print('predict2')
    pred2 = None
    counter = 0
    for mp in model_path2:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred2 is None:
            pred2 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred2 = pred2 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred2 = pred2 / counter
    
    pred[:,config.TG_MY_MODEL2] = (pred[:,config.TG_MY_MODEL2] + pred2[:,config.TG_MY_MODEL2]) * 0.5
    
    """
    # prediction3
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path))
    print('predict3')
    pred3 = None
    counter = 0
    for mp in model_path3_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred3 is None:
            pred3 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred3 = pred3 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred3 = pred3 / counter
    pred3 = pred3 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL3_CLASS] = pred3[:,config.TG_MY_MODEL3_CLASS]
    """
    
    # prediction4
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path), num_class=config.NUMCLASS_MODEL4_CLASS)
    print('predict4')
    pred4 = None
    counter = 0
    for mp in model_path4_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred4 is None:
            pred4 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred4 = pred4 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred4 = pred4 / counter
    pred4 = pred4 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL4_CLASS] = pred4[:,config.TG_MY_MODEL4_CLASS]
    
    # prediction5
    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))
    print('predict5')
    pred5 = None
    counter = 0
    for mp in model_path5:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred5 is None:
            pred5 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred5 = pred5 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred5 = pred5 / counter
    
    pred[:,config.TG_MY_MODEL5] = (pred[:,config.TG_MY_MODEL5] + pred5[:,config.TG_MY_MODEL5]) * 0.5
    
    
    # predcition by rule base
    print('predcition by rule base')
    pred_question_type_spelling = calc_pred_question_type_spelling(test_df)
    pred[:,19] = pred_question_type_spelling

    # post processing to 0 or 1
    print('post processing to 0 or 1')
    pred = post_proc_to0or1(pred)
    
    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

        # predcition by rule base
        pred_question_type_spelling = calc_pred_question_type_spelling(train_df)
        pred[:,19] = pred_question_type_spelling
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [13]:
def run_20021102():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_2_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_2_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_2_model_fold2'),
        ]
    
    model_path2 = [
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold0'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold1'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold2'),
        ]
    
    #model_path3_class = [
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold0'),
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold1'),
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold2'),
    #    ]
    
    model_path4_class = [
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold0'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold1'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold2'),
        ]
    
    model_path5 = [
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold0'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold1'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold2'),
        ]


    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    
    # prediction
    model = Bert_v1_2(BertUtils.get_bert_model(pretrained_model_path))
    print('predict')
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    
    # prediction2
    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))
    print('predict2')
    pred2 = None
    counter = 0
    for mp in model_path2:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred2 is None:
            pred2 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred2 = pred2 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred2 = pred2 / counter
    
    pred[:,config.TG_MY_MODEL2] = (pred[:,config.TG_MY_MODEL2] + pred2[:,config.TG_MY_MODEL2]) * 0.5
    
    """
    # prediction3
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path))
    print('predict3')
    pred3 = None
    counter = 0
    for mp in model_path3_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred3 is None:
            pred3 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred3 = pred3 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred3 = pred3 / counter
    pred3 = pred3 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL3_CLASS] = pred3[:,config.TG_MY_MODEL3_CLASS]
    """
    
    # prediction4
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path), num_class=config.NUMCLASS_MODEL4_CLASS)
    print('predict4')
    pred4 = None
    counter = 0
    for mp in model_path4_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred4 is None:
            pred4 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred4 = pred4 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred4 = pred4 / counter
    pred4 = pred4 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL4_CLASS] = pred4[:,config.TG_MY_MODEL4_CLASS]
    
    # prediction5
    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))
    print('predict5')
    pred5 = None
    counter = 0
    for mp in model_path5:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred5 is None:
            pred5 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred5 = pred5 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred5 = pred5 / counter
    
    pred[:,config.TG_MY_MODEL5] = (pred[:,config.TG_MY_MODEL5] + pred5[:,config.TG_MY_MODEL5]) * 0.5
    
    
    # predcition by rule base
    print('predcition by rule base')
    pred_question_type_spelling = calc_pred_question_type_spelling(test_df)
    pred[:,19] = pred_question_type_spelling

    # post processing to 0 or 1
    print('post processing to 0 or 1')
    pred = post_proc_to0or1(pred)
    
    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

        # predcition by rule base
        pred_question_type_spelling = calc_pred_question_type_spelling(train_df)
        pred[:,19] = pred_question_type_spelling
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [14]:
def run_20021103():
    DO_CHECK = False

    print('model')
    pretrained_model = config.BERT_PRETRAINED_MODEL
    pretrained_tokenizer_path = config.BERT_TOKENIZER_PATH
    pretrained_model_path = config.BERT_PRETRAINED_MODEL_PATH
    max_sequence_length = 512

    tokenizer = BertUtils.get_bert_tokenizer(pretrained_tokenizer_path)
    
    model_path = [
        os.path.join(config.MY_MODEL, 'Bert_v1_2_model_fold0'),
        os.path.join(config.MY_MODEL, 'Bert_v1_2_model_fold1'),
        os.path.join(config.MY_MODEL, 'Bert_v1_2_model_fold2'),
        ]
    
    model_path1 = [
        os.path.join(config.MY_MODEL1, 'Bert_v1_0_model_fold0'),
        os.path.join(config.MY_MODEL1, 'Bert_v1_0_model_fold1'),
        os.path.join(config.MY_MODEL1, 'Bert_v1_0_model_fold2'),
        os.path.join(config.MY_MODEL1, 'Bert_v1_0_model_fold3'),
        os.path.join(config.MY_MODEL1, 'Bert_v1_0_model_fold4'),
        ]
    
    model_path2 = [
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold0'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold1'),
        os.path.join(config.MY_MODEL2, 'Bert_v1_0_comb2_model_fold2'),
        ]
    
    #model_path3_class = [
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold0'),
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold1'),
    #    os.path.join(config.MY_MODEL3_CLASS, 'Bert_v1_0_clasify_comb1_model_fold2'),
    #    ]
    
    model_path4_class = [
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold0'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold1'),
        os.path.join(config.MY_MODEL4_CLASS, 'Bert_v1_0_clasify_comb2_model_fold2'),
        ]
    
    model_path5 = [
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold0'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold1'),
        os.path.join(config.MY_MODEL5, 'Bert_v1_0_comb1_model_fold2'),
        ]


    print('test data')
    # raw data
    test_df = InputData.get_test_data()
    # bert input
    test_bert_inp = BertData_v3.compute_input_arrays(test_df, tokenizer, max_sequence_length)
    
    # prediction
    model = Bert_v1_2(BertUtils.get_bert_model(pretrained_model_path))
    print('predict')
    pred = None
    counter = 0
    for mp in model_path:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred is None:
            pred = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred = pred / counter
    
    # prediction1
    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))
    print('predict')
    pred1 = None
    counter = 0
    for mp in model_path1:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred1 is None:
            pred1 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred1 = pred1 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred1 = pred1 / counter
    pred = (pred + pred1) * 0.5
    
    
    # prediction2
    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))
    print('predict2')
    pred2 = None
    counter = 0
    for mp in model_path2:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred2 is None:
            pred2 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred2 = pred2 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred2 = pred2 / counter
    
    #pred[:,config.TG_MY_MODEL2] = (pred[:,config.TG_MY_MODEL2] + pred2[:,config.TG_MY_MODEL2]) * 0.5
    pred[:,config.TG_MY_MODEL2] = (pred[:,config.TG_MY_MODEL2] * 2 + pred2[:,config.TG_MY_MODEL2] * 1) / 3

    
    """
    # prediction3
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path))
    print('predict3')
    pred3 = None
    counter = 0
    for mp in model_path3_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred3 is None:
            pred3 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred3 = pred3 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred3 = pred3 / counter
    pred3 = pred3 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL3_CLASS] = pred3[:,config.TG_MY_MODEL3_CLASS]
    """
    
    # prediction4
    model_cls = Bert_v1_0_classify(BertUtils.get_bert_model(pretrained_model_path), num_class=config.NUMCLASS_MODEL4_CLASS)
    print('predict4')
    pred4 = None
    counter = 0
    for mp in model_path4_class:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model_cls.load_state_dict(torch.load(mp))

        if pred4 is None:
            pred4 = calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        else:
            pred4 = pred4 + calc_pred_Bert_v1_0(model_cls, loader, with_target=False, classify=True)
        counter += 1
    pred4 = pred4 / counter
    pred4 = pred4 / 100 # convert value to less than 1
    
    pred[:,config.TG_MY_MODEL4_CLASS] = pred4[:,config.TG_MY_MODEL4_CLASS]
    
    # prediction5
    model = Bert_v1_0(BertUtils.get_bert_model(pretrained_model_path))
    print('predict5')
    pred5 = None
    counter = 0
    for mp in model_path5:
        ds = QADataset_SeparateQA(test_bert_inp[0], test_bert_inp[1], test_bert_inp[2], 
                                  test_bert_inp[3], test_bert_inp[4], test_bert_inp[5])

        batch_size = 32
        loader = get_dataloader(ds, batch_size, shuffle=False)

        model.load_state_dict(torch.load(mp))

        if pred5 is None:
            pred5 = calc_pred_Bert_v1_0(model, loader, with_target=False)
        else:
            pred5 = pred5 + calc_pred_Bert_v1_0(model, loader, with_target=False)
        counter += 1
    pred5 = pred5 / counter
    
    #pred[:,config.TG_MY_MODEL5] = (pred[:,config.TG_MY_MODEL5] + pred5[:,config.TG_MY_MODEL5]) * 0.5
    pred[:,config.TG_MY_MODEL5] = (pred[:,config.TG_MY_MODEL5] * 2 + pred5[:,config.TG_MY_MODEL5] * 1) / 3
    
    
    
    # predcition by rule base
    print('predcition by rule base')
    pred_question_type_spelling = calc_pred_question_type_spelling(test_df)
    pred[:,19] = pred_question_type_spelling

    # post processing to 0 or 1
    print('post processing to 0 or 1')
    pred = post_proc_to0or1(pred)
    
    # make submission
    print('submission')
    Submission.make_submission(pred)

    if DO_CHECK:
        print('train data')
        # raw data
        train_df = InputData.get_train_data()
        # bert input
        train_bert_inp = BertData_v3.compute_input_arrays(train_df, tokenizer, max_sequence_length)
        train_bert_label = BertData_v3.compute_output_arrays(train_df) # 30
        # prediction
        pred = None
        counter = 0
        for mp in model_path:
            ds = QADataset_SeparateQA(train_bert_inp[0], train_bert_inp[1], train_bert_inp[2], 
                                      train_bert_inp[3], train_bert_inp[4], train_bert_inp[5],
                                      train_bert_label)

            batch_size = 32
            loader = get_dataloader(ds, batch_size, shuffle=False)

            model.load_state_dict(torch.load(mp))

            if pred is None:
                pred = calc_pred_Bert_v1_0(model, loader, with_target=True)
            else:
                pred = pred + calc_pred_Bert_v1_0(model, loader, with_target=True)
            counter += 1
        pred = pred / counter
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

        # predcition by rule base
        pred_question_type_spelling = calc_pred_question_type_spelling(train_df)
        pred[:,19] = pred_question_type_spelling
        # make submission
        score = compute_spearmanr(train_bert_label, pred)
        print('train data score : {0}'.format(score))

    return

In [15]:
#run_20020401()
#run_20020601()
#run_200208801()
#run_200208802()
#run_200208803()
#run_20021001()
#run_20021002()
#run_20021003()
#run_20021101()
#run_20021102()
run_20021103()

model
test data
predict



predict



predict2



predict4



predict5



predcition by rule base
calc_pred_question_type_spelling
64
7
What word describes the action of changing the grammatical person of a sentence?
Positive way of saying emotional appeal?
"Report by" or "report from"
What is the correct possessive form of "One of the guys"?
post processing to 0 or 1
submission
